Forecast the next month(s) for each Team Member, both for individual clients and total hours.

In [ ]:
# Import libraries
import pandas as pd
from datetime import datetime
import matplotlib.pylab as plt
from plotnine import ggplot, aes, geom_line
import statsmodels.api as sm
import numpy as np
from statsmodels.graphics import tsaplots
from statsmodels.tsa.arima.model import ARIMA
import hts

%matplotlib inline


# Read Data
df = pd.read_csv('data/dataset.csv')
df.head()

In [ ]:
# Update Data Types
df['Data Type'] = df['Data Type'].astype('category')
df['Month'] = pd.to_datetime(df['Month'])
df['Client'] = df['Client'].astype('category')
df['Responsible'] = df['Responsible'].astype('category')
df.info()

In [ ]:
# Exploratory Analysis

def get_column_description(data):
    for col in data.columns:
        if data[col].dtype == 'category':
            print(f'{col} unique values: {len(data[col].unique())}\n')
        elif data[col].dtype == 'datetime64[ns]':
            print(f'Min Date: {data[col].min()}')
            print(f'Max Date: {data[col].max()}\n')
        else:
            print(f'{col}:\n{df[col].describe()}\n')

get_column_description(df)

In [ ]:
df.groupby(['Responsible', 'Client']).count().rename(columns={'Data Type': 'Count'}).sort_values(by='Count', ascending=False).Count

In [ ]:
# Example of Team member with all 13 months with one client
example = df.loc[(df.Responsible == 'Team Member 12') & (df.Client == 'Client 18')].sort_values(by='Month')
example.index = pd.DatetimeIndex(example.Month, freq=pd.DatetimeIndex(example.Month).inferred_freq)

display(example)

(
    ggplot(example)
    + aes(x='Month', y='Est Hrs')
    + geom_line()
)

In [ ]:
# Because there aren't a complete 13-month range for each team member, do some pre-processing filling in empty months with zero

def fill_empty_dates(data):
    unique_pairs = data[['Responsible', 'Client']].drop_duplicates().sort_values(by=['Responsible', 'Client'])
    dates = pd.concat([pd.DataFrame({'Month':pd.date_range(datetime(2021, 1, 1), periods=13, freq='1M') - pd.offsets.MonthBegin(1)})]*len(unique_pairs), ignore_index=True)
    resp_cli = pd.concat([unique_pairs]*13, ignore_index=True).sort_values(by=['Responsible', 'Client'])
    out = pd.DataFrame({
        'Month':dates.Month,
        'Responsible':resp_cli.Responsible,
        'Client':resp_cli.Client
    }).sort_values(by=['Responsible', 'Client', 'Month']).reset_index(drop=True)

    out = pd.merge(out, data.drop(columns='Data Type', axis=1), how='left', on=['Month', 'Responsible', 'Client'])
    out.loc[out['Est Hrs'].isna(), 'Est Hrs'] = 0
    return out

df = fill_empty_dates(df)


In [ ]:
# Statistical tests/EDA - whether data is stationary, granger-causality, distributions, visualizations

# One of the challenges with this dataset is there is only one "cycle" of data (only one complete year). 
# You typically need at least two complete cycles of data in order to infer any seasonality. However,
# we can still make an effort to ensure the data is stationary (not a trend in the data) prior to 
# using any forecast methods where stationary data is a constraint.


# Make data stationary by using the `diff()` function
example['res'] = example['Est Hrs'].diff()

(
    ggplot(example)
    + aes(x='Month', y='res')
    + geom_line()
)

In [ ]:
ac_plt = tsaplots.plot_acf(example.res.dropna(), lags = 1)

In [ ]:
# Naive Baseline of Example - a quick example of the "Score to beat"

mod = ARIMA(example.res, order=(0,1,0))
res = mod.fit()

print(res.summary())

example['y_hat'] = res.predict()
example.loc[datetime(2021,2,1), 'y_hat'] = example['Est Hrs'].iloc[0]
example['y_hat'] = example['y_hat'].cumsum()
(
    ggplot(example[['Month', 'Est Hrs', 'y_hat']].melt(id_vars='Month'))
    + aes(x='Month', y='value', color='variable')
    + geom_line()
)

In [ ]:
mod = ARIMA(example.res, order=(1,1,0))
res = mod.fit()

print(res.summary())

example['y_hat'] = res.predict()
example.loc[datetime(2021,2,1), 'y_hat'] = example['Est Hrs'].iloc[0]
example['y_hat'] = example['y_hat'].cumsum()
(
    ggplot(example[['Month', 'Est Hrs', 'y_hat']].melt(id_vars='Month'))
    + aes(x='Month', y='value', color='variable')
    + geom_line()
)

In [ ]:
# Make all data stationary
df['res'] = df.groupby(['Responsible', 'Client'])['Est Hrs'].diff()

In [ ]:
# Naive Baseline (Random Walk)

def get_pred(data, pred, col):
    data = data.reset_index(drop=True)
    pred[1] = data.iloc[0]['Est Hrs']
    data[col] = pred.cumsum().values
    return data

def naive_baseline(data):
    data.index = pd.DatetimeIndex(data.Month, freq=pd.DatetimeIndex(data.Month).inferred_freq)
    mod = ARIMA(data.res, order=(0,1,0), freq='MS')
    data = get_pred(data, mod.fit().predict(), 'baseline')
    return data

#nb = df.groupby(['Responsible', 'Client'], as_index=False, observed=True).apply(func=naive_baseline).reset_index(drop=True)

In [ ]:
# Heirarchal - Each client is a part of the whole for each Team Member; Each Team member is part of the overall Team

def get_hierarchy(data, lvl1, lvl2):
    new_ = f'{lvl1}_{lvl2}'
    data[new_] = data.apply(lambda x: f'{x[lvl1]}_{x[lvl2]}', axis=1)
    l1s = data[lvl1].unique()
    l2s = data[new_].unique()
    total = {'total': list(l1s)}
    l1 = {k: [v for v in l2s if k == v.split('_')[0]] for k in l1s}
    hier = {**total, **l1}
    return hier, new_

def get_hierarchal(data, lvl1, lvl2, date_col='Month', val='Est Hrs'):
    hier, new_ = get_hierarchy(data, lvl1, lvl2)
    hd = data.pivot(index=date_col, columns=new_, values=val)\
        .join(
            data.groupby([date_col, lvl1], as_index=False, observed=True)\
                .agg({val : lambda x: data.loc[x.index][val].sum()})\
                    .pivot(index=date_col, columns=lvl1, values=val)
            )\
                .join(
                    data.groupby(date_col, observed=True)\
                        .agg({val : lambda x: data.loc[x.index][val].sum()})\
                            .rename(columns={val:'total'})
                    )
    return hier, hd

hierarchy, hierarchy_df = get_hierarchal(df, 'Responsible', 'Client')   

In [ ]:
def hier_arima(hdf, col, order=(1,1,0)):
    mod = ARIMA(hdf[col].diff(), order=order, freq='MS')
    mod = mod.fit()
    #return mod.predict()
    return mod

    #example['y_hat'] = res.predict()
    #example.loc[datetime(2021,2,1), 'y_hat'] = example['Est Hrs'].iloc[0]
    #example['y_hat'] = example['y_hat'].cumsum()
mod = hier_arima(hierarchy_df, 'total')
mod.fitted_values()

In [ ]:
hw = hts.HTSRegressor(model='holt_winters', revision_method='OLS', n_jobs=1)
hw = hw.fit(hierarchy_df, nodes=hierarchy)
hw_pred = reg.predict(steps_ahead=1)

auto_arima = hts.HTSRegressor(model='auto_arima', revision_method='OLS', n_jobs=1)
auto_arima = auto_arima.fit(hierarchy_df, nodes=hierarchy)
auto_arima_pred = auto_arima.predict(steps_ahead=1)

In [ ]:
auto_arima_pred

In [ ]:
# Random Forest